# Тестовое задание для компании SCID
## Описание задачи
1. Извлечение текста из PDF-документов:
   - Используя библиотеку PyPDF2, pdfminer или PyMuPDF, реализовать функцию, которая принимает на вход путь к PDF-файлу и возвращает извлеченный текст.

2. Предобработка текста:
   - Реализовать функцию для предобработки извлеченного текста. Включить в нее следующие шаги:
     - Удаление специальных символов и лишних пробелов.
     - Токенизация текста (разделение на слова).

3. Обучение модели:
   - С использованием библиотеки Keras или PyTorch создать простую нейронную сеть для классификации текста. Для обучения использовать набор данных, содержащий тексты и соответствующие метки (например, категории документов). В работе будет использован датасет 20 Newsgroups.

4. Интерфейс для пользователя:
   - Реализовать простой интерфейс командной строки или веб-интерфейс с использованием Flask, который позволит пользователю загрузить PDF-документ и получить предсказанную категорию документа.

5. Документация:
   - Подготовить к проекту документацию, описывающую структуру кода, инструкции по установке и запуску, а также примеры использования.

Дополнительные требования:
- Используйте Git для контроля версий вашего проекта.
- Напишите тесты для ключевых функций вашего кода.
- Обратите внимание на обработку ошибок (например, что происходит при загрузке некорректного PDF).

## Import dependencies

In [1]:
import numpy as np
import fitz
import re
import nltk
import joblib
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

In [2]:
# Функция для выделения текста из .pdf
def extract_text(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [3]:
# Функция для токенизации
def tokenize_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = nltk.word_tokenize(text.lower(), language='english')
    return ' '.join(tokens)

In [5]:
# Загрузка и подготовка данных
data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
texts = data['data']
labels = data['target']

# Очистка текста от лишних символов
texts = [tokenize_text(text) for text in texts]

# Разделение на выборки
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


# Векторизация выборок
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [10]:
# Обучаем модель
model = keras.Sequential([
    keras.layers.Input(shape=(X_train_vec.shape[1],)),
    keras.layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.0000001)),
    keras.layers.Dense(20, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_vec.toarray(), y_train, epochs=5, validation_data=(X_test_vec.toarray(), y_test))

Epoch 1/5
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3818 - loss: 2.5648 - val_accuracy: 0.6613 - val_loss: 1.3475
Epoch 2/5
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7593 - loss: 1.0394 - val_accuracy: 0.6809 - val_loss: 1.0841
Epoch 3/5
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8365 - loss: 0.6837 - val_accuracy: 0.6878 - val_loss: 1.0306
Epoch 4/5
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8957 - loss: 0.4802 - val_accuracy: 0.6870 - val_loss: 1.0354
Epoch 5/5
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9260 - loss: 0.3611 - val_accuracy: 0.6822 - val_loss: 1.0645


Простая нейросеть с одним скрытым слоем обладает высокой метрикой 0.92 на тренировочной выборке и относительно невысокой метрикой 0.68 на тестовой выборке, что говорит о переобучении модели, но для такой простой модели с таким сроком обучения результат нормальный. Модель вполне можно применять для классификации текстов.

In [16]:
# Сохраняем предобученный векторизатор и модель
joblib.dump(vectorizer, 'my_vectorizer.pkl')
model.save('my_model.keras')


# Нам понадобится файл с сохраненными категориями, поэтому соберем его
categories = data.target_names
with open('categories.json', 'w', encoding='utf-8') as f:
    json.dump(categories, f, ensure_ascii=False, indent=2)